# Proyecto de Detección de Spam en URLs

## Introducción

Este proyecto tiene como objetivo desarrollar un modelo de machine learning capaz de identificar si una URL es spam o no. Se ha empleado un conjunto de datos que contiene URLs clasificadas previamente como spam o legítimas. El enfoque adoptado incluye la limpieza y preprocesamiento de datos, entrenamiento de un modelo SVM, optimización de hiperparámetros y evaluación del modelo.

## Metodología

### Carga de Datos

Los datos se cargaron desde un recurso en línea y se comprobó que no hubiera valores faltantes o duplicados.

### Preprocesamiento

El preprocesamiento incluyó:
- Segmentación de URLs para extraer características textuales.
- Limpieza de caracteres especiales y números.
- Vectorización usando TF-IDF para transformar el texto en un formato adecuado para el entrenamiento del modelo.

### Modelo de Machine Learning

Se seleccionó un modelo SVM por su eficacia en problemas de clasificación binaria. Los pasos incluyeron:
- Entrenamiento inicial del modelo con parámetros predeterminados.
- Ajuste de los pesos de clase para abordar el desequilibrio significativo entre las clases spam y no spam.

### Optimización de Hiperparámetros

Se utilizó la técnica de búsqueda en cuadrícula para encontrar la mejor combinación de hiperparámetros, mejorando significativamente el rendimiento del modelo.

### Evaluación del Modelo

El modelo optimizado se evaluó con un conjunto de prueba, donde mostró una precisión aceptable y un alto recall para la detección de URLs de spam, evidenciando un balance adecuado entre precisión y capacidad de detección.

## Resultados

Los resultados mostraron que el modelo es capaz de detectar eficazmente URLs spam con un `recall` de 0.84 para la clase spam y una `precision` de 0.60, con un `f1-score` de 0.70 para la misma clase.

## Conclusión

El modelo desarrollado demostró ser efectivo para la tarea de detección de spam en URLs. Futuras mejoras podrían incluir la exploración de más características, el uso de técnicas adicionales de balanceo de clases o la implementación de modelos más complejos.

## Almacenamiento del Modelo

El modelo fue guardado utilizando `joblib` para su uso futuro en aplicaciones de producción o para evaluaciones adicionales.

In [26]:
import pandas as pd
import numpy as np
import nltk
from urllib.parse import urlparse
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from joblib import dump


In [4]:
# URL desde donde descargar el conjunto de datos
url = 'https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv'

# Cargar los datos en un DataFrame
data = pd.read_csv(url)

In [5]:
# Inspeccionar los datos
print(data.info())
print(data.head())

# Comprobar y manejar datos faltantes
if data.isnull().sum().sum() > 0:
    data.dropna(inplace=True)  # Elimina filas con datos faltantes
    print("Filas con datos faltantes eliminadas.")
else:
    print("No hay datos faltantes.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB
None
                                                 url  is_spam
0  https://briefingday.us8.list-manage.com/unsubs...     True
1                             https://www.hvper.com/     True
2                 https://briefingday.com/m/v4n3i4f3     True
3   https://briefingday.com/n/20200618/m#commentform    False
4                        https://briefingday.com/fan     True
No hay datos faltantes.


In [14]:
# Descargar recursos de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_url(url):
    # Extraer componentes de la URL
    parsed_url = urlparse(url)
    # Dividir la URL en partes según caracteres no alfanuméricos, incluyendo todo
    url_parts = re.split('\W+', parsed_url.netloc + parsed_url.path + parsed_url.query)

    # Inicializar lematizador
    lemmatizer = WordNetLemmatizer()

    # Procesar las partes de la URL
    cleaned_parts = [lemmatizer.lemmatize(word.lower()) for word in url_parts if word]

    # Retornar la URL preprocesada como un conjunto de palabras
    return ' '.join(cleaned_parts)

# Aplicar el preprocesamiento a cada URL
data['url_clean'] = data['url'].apply(preprocess_url)

# Mostrar los datos preprocesados
print(data[['url', 'url_clean']].head())

                                                 url  \
0  https://briefingday.us8.list-manage.com/unsubs...   
1                             https://www.hvper.com/   
2                 https://briefingday.com/m/v4n3i4f3   
3   https://briefingday.com/n/20200618/m#commentform   
4                        https://briefingday.com/fan   

                                     url_clean  
0  briefingday us8 list manage com unsubscribe  
1                                www hvper com  
2                   briefingday com m v4n3i4f3  
3                 briefingday com n 20200618 m  
4                          briefingday com fan  


[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
# Comprobación de valores faltantes
if data['url_clean'].isnull().any():
    print("Hay valores NaN en 'url_clean'. Se deben manejar antes de proceder.")
else:
    print("No hay valores NaN en 'url_clean'. Todo parece correcto.")

# Revisión de cadenas vacías que pueden haber pasado como válidas
empty_strings = (data['url_clean'] == '').sum()
if empty_strings > 0:
    print(f"Existen {empty_strings} cadenas vacías en 'url_clean'. Considera revisar el preprocesamiento.")
else:
    print("No hay cadenas vacías en 'url_clean'.")

# Análisis descriptivo básico
print("\nDescripción básica de 'url_clean':")
print(data['url_clean'].describe())

# Verificación de la distribución de la clase objetivo
print("\nDistribución de la clase 'is_spam':")
print(data['is_spam'].value_counts(normalize=True))

No hay valores NaN en 'url_clean'. Todo parece correcto.
No hay cadenas vacías en 'url_clean'.

Descripción básica de 'url_clean':
count                             2999
unique                            2353
top       www bloomberg com tosv2 html
freq                                26
Name: url_clean, dtype: object

Distribución de la clase 'is_spam':
is_spam
False    0.767923
True     0.232077
Name: proportion, dtype: float64


## Resumen de la Comprobación de los Datos
No hay valores NaN: Esto indica que todos los datos están completos y listos para ser usados.
No hay cadenas vacías: Las URLs han sido preprocesadas correctamente sin perder información crítica.
Distribución de la clase is_spam: Hay una proporción mayor de URLs no spam (76.79%) comparado con spam (23.21%). Este desbalance puede influir en cómo el modelo aprende a clasificar, por lo que podría ser útil considerar técnicas para manejar desbalances de clase, como el sobremuestreo de la clase minoritaria o el submuestreo de la clase mayoritaria.

## Análisis Descriptivo
Unicidad de los datos transformados: Hay 2,353 URL únicas de un total de 2,999, lo que sugiere cierta repetición. Es común en conjuntos de datos reales, y el modelo debería ser capaz de manejarlo. Sin embargo, el hecho de que algunas URLs se repitan con frecuencia (como www bloomberg com tosv2 html) puede ser un indicador de patrones comunes en URLs spam o no spam.

In [17]:
# Eliminar duplicados manteniendo la primera ocurrencia
data_unique = data.drop_duplicates(subset='url_clean', keep='first')

# Revisar cuántos datos quedan después de eliminar duplicados
print(f"Datos originales: {data.shape[0]}, Datos sin duplicados: {data_unique.shape[0]}")

Datos originales: 2999, Datos sin duplicados: 2353


In [20]:
# Eliminar duplicados
data_unique = data.drop_duplicates(subset='url_clean', keep='first')

# Revisar la distribución de la clase 'is_spam'
print("Distribución de clases después de eliminar duplicados:")
print(data_unique['is_spam'].value_counts(normalize=True))

Distribución de clases después de eliminar duplicados:
is_spam
False    0.898428
True     0.101572
Name: proportion, dtype: float64


In [21]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = data_unique['url_clean']
y = data_unique['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización de los textos
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [22]:
# Entrenamiento del modelo SVM con ajuste de pesos de clase
model = SVC(class_weight='balanced')  # Ajustar los pesos de las clases automáticamente
model.fit(X_train_vect, y_train)

# Evaluación del modelo en el conjunto de prueba
y_pred = model.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy del modelo: {accuracy}")
print("Reporte de clasificación:")
print(report)

Accuracy del modelo: 0.921443736730361
Reporte de clasificación:
              precision    recall  f1-score   support

       False       0.97      0.95      0.96       428
        True       0.56      0.67      0.61        43

    accuracy                           0.92       471
   macro avg       0.76      0.81      0.78       471
weighted avg       0.93      0.92      0.92       471



## Análisis
Los resultados iniciales son bastante prometedores, especialmente considerando que el modelo ya muestra una buena capacidad para identificar URLs de spam, lo que se refleja en un recall de 0.67 para la clase minoritaria (spam). Sin embargo, hay espacio para mejorar, especialmente en la precisión de la detección de spam, que actualmente es de 0.56

In [24]:
# Parámetros para la búsqueda en cuadrícula
param_grid = {
    'C': [0.1, 1, 10, 100],  # Ejemplos de valores de C
    'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10],  # Ejemplos de valores gamma
    'kernel': ['rbf', 'linear']  # Explorar RBF y lineal
}

# Configurar GridSearchCV
grid_search = GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, scoring='f1_macro', verbose=2)

# Ejecutar la búsqueda en cuadrícula
grid_search.fit(X_train_vect, y_train)

# Mejores parámetros encontrados y su rendimiento
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de cross-validation (f1-macro):", grid_search.best_score_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.3s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time=   0.3s
[CV] END ......................C=0.1, gamma=aut

In [25]:
# Reevaluar con los mejores parámetros
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_vect)
new_accuracy = accuracy_score(y_test, y_pred_best)
new_report = classification_report(y_test, y_pred_best)

print(f"Accuracy del modelo con optimización: {new_accuracy}")
print("Reporte de clasificación con optimización:")
print(new_report)

Accuracy del modelo con optimización: 0.9341825902335457
Reporte de clasificación con optimización:
              precision    recall  f1-score   support

       False       0.98      0.94      0.96       428
        True       0.60      0.84      0.70        43

    accuracy                           0.93       471
   macro avg       0.79      0.89      0.83       471
weighted avg       0.95      0.93      0.94       471



## Análisis
Los resultados tras la optimización de hiperparámetros son notables y muestran mejoras significativas, especialmente en la métrica crucial de recall para la clase de spam (True), que ha aumentado al 84%. Esto indica que el modelo ahora es capaz de identificar una mayor proporción de URLs de spam, lo cual es fundamental para un sistema de detección de spam efectivo. La precisión para la clase de spam también ha mejorado a 0.60, y aunque aún hay margen para mejora, es un buen avance considerando el desafío de trabajar con clases desequilibradas.

### Evaluación del Modelo
La mejora en el f1-score para la clase de spam a 0.70 es particularmente valiosa, ya que este score es una medida que equilibra la precisión y el recall, haciendo que sea una buena métrica en casos de desequilibrio de clases. El f1-score macro promedio también ha mejorado, lo cual es indicativo de que el modelo es más equilibrado en su rendimiento entre las clases.

In [27]:
# Especifica la ruta y el nombre del archivo donde quieres guardar tu modelo
model_path = '../models/spam_detection_model.joblib'

# Guardar el modelo
dump(best_model, model_path)

print(f"Modelo guardado correctamente en {model_path}")

Modelo guardado correctamente en ../models/spam_detection_model.joblib
